In [ ]:
import os
import json
import kagglehub



# Now login
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
from pathlib import Path
import kagglehub

# Login is assumed to be done already
binshilin_genreal_old_deepfakes_path = kagglehub.dataset_download('binshilin/poc-final-dataset-vv')

print('Data source import complete.')
dataset_path = Path(binshilin_genreal_old_deepfakes_path)
print(f'Dataset path: {str(dataset_path)}')


100%|██████████| 155M/155M [00:05<00:00, 30.4MB/s]

Extracting files...


Data source import complete.
Dataset path: /root/.cache/kagglehub/datasets/binshilin/poc-final-dataset-vv/versions/1


In [3]:
! pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=c7807df314e87db4be43899bcfeedb2100f871a077b90251d95700e284b34f1b
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet_pytorch


In [4]:
# Training with flattened dataset (Person_001, Person_002 all have come outside)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import numpy as np
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
import warnings
warnings.filterwarnings("ignore")

# ----------- Dataset Class with Sequential Batching -----------
class FaceSequenceDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_frames=32, min_frames=16, frame_interval=1):
        self.samples = []
        self.transform = transform
        self.label_map = {'original': 0, 'fake': 1}
        self.max_frames = max_frames
        self.min_frames = min_frames
        self.frame_interval = frame_interval

        print(f"Scanning dataset in: {root_dir}")
        print(f"Using max_frames={max_frames}, min_frames={min_frames}")

        for label in ['original', 'fake']:
            class_dir = os.path.join(root_dir, label)
            if not os.path.exists(class_dir):
                print(f"Warning: {class_dir} does not exist, skipping...")
                continue

            print(f"Processing {label} class...")
            folder_count = 0

            # Iterate through video folders directly under original/fake
            for folder_name in os.listdir(class_dir):
                folder_path = os.path.join(class_dir, folder_name)
                if not os.path.isdir(folder_path):
                    continue

                folder_count += 1

                # Get all frame files in this video folder
                frame_paths = []
                for ext in ['*.jpg', '*.jpeg', '*.png']:
                    frame_paths.extend(glob.glob(os.path.join(folder_path, ext)))

                # Sort frames by filename to maintain temporal order
                frame_paths = sorted(frame_paths)

                if len(frame_paths) == 0:
                    print(f"Warning: No frames found in {folder_path}")
                    continue

                # Create sequential batches from this video's frames
                batches_created = self._create_sequential_batches(frame_paths, self.label_map[label], folder_name)

                if folder_count % 100 == 0:
                    print(f"Processed {folder_count} {label} folders, created {len(self.samples)} total batches so far")

            print(f"Finished processing {label} class: {folder_count} folders")

    def _create_sequential_batches(self, frame_paths, label, folder_name):
        """Create sequential batches of frames from a single video sequence"""
        total_frames = len(frame_paths)
        batches_created = 0

        if total_frames < self.min_frames:
            print(f"Skipping {folder_name}: only {total_frames} frames (< {self.min_frames})")
            return batches_created

        # Create overlapping or non-overlapping batches
        # Using stride = max_frames for non-overlapping batches
        # You can change this to max_frames//2 for overlapping batches
        stride = self.max_frames  # Non-overlapping batches

        start_idx = 0
        while start_idx < total_frames:
            end_idx = min(start_idx + self.max_frames, total_frames)
            batch_frames = frame_paths[start_idx:end_idx]

            # Only keep batches that meet minimum frame requirement
            if len(batch_frames) >= self.min_frames:
                self.samples.append((batch_frames, label, folder_name))
                batches_created += 1

            start_idx += stride

            # If remaining frames are less than min_frames, break
            if total_frames - start_idx < self.min_frames:
                break

        return batches_created

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        frame_paths, label, folder_name = self.samples[idx]

        images = []
        successful_loads = 0

        for frame_path in frame_paths:
            try:
                img = Image.open(frame_path).convert('RGB')
                if self.transform:
                    img = self.transform(img)
                images.append(img)
                successful_loads += 1
            except Exception as e:
                print(f"Error loading {frame_path}: {e}")
                # Use a black image as fallback
                if self.transform:
                    fallback_img = Image.new('RGB', (224, 224), (0, 0, 0))
                    img = self.transform(fallback_img)
                else:
                    img = torch.zeros(3, 224, 224)
                images.append(img)

        # Pad sequence to max_frames if needed
        while len(images) < self.max_frames:
            # Duplicate the last frame for padding
            if len(images) > 0:
                images.append(images[-1].clone())
            else:
                # Fallback if no images were loaded successfully
                if self.transform:
                    fallback_img = Image.new('RGB', (224, 224), (0, 0, 0))
                    img = self.transform(fallback_img)
                else:
                    img = torch.zeros(3, 224, 224)
                images.append(img)

        images = torch.stack(images)  # (T, C, H, W)

        # Create a mask for valid frames (non-padded)
        valid_frames = min(len(frame_paths), self.max_frames)

        return images, torch.tensor(label, dtype=torch.long), valid_frames

# ----------- Improved Model with Proper Spatial-Temporal Processing -----------
class SpatialTemporalDeepfakeDetector(nn.Module):
    def __init__(self, num_classes=2, max_frames=32, feature_dim=1408):
        super().__init__()

        # Spatial feature extractor
        self.spatial_encoder = EfficientNet.from_pretrained('efficientnet-b2')
        self.spatial_encoder._fc = nn.Identity()

        # Spatial feature projection
        self.spatial_projection = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Temporal modeling with LSTM + Transformer
        self.temporal_lstm = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )

        # Temporal attention
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=512,  # bidirectional LSTM output
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )

        # Positional encoding for temporal sequences
        self.pos_encoding = nn.Parameter(torch.randn(max_frames, 512))

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x, valid_frames=None):  # x: (B, T, C, H, W)
        B, T, C, H, W = x.shape

        # Extract spatial features for each frame
        x_reshaped = x.view(B * T, C, H, W)
        spatial_features = self.spatial_encoder.extract_features(x_reshaped)  # (B*T, 1408, h, w)

        # Project spatial features
        spatial_features = self.spatial_projection(spatial_features)  # (B*T, 512)
        spatial_features = spatial_features.view(B, T, -1)  # (B, T, 512)

        # Add positional encoding
        spatial_features = spatial_features + self.pos_encoding[:T].unsqueeze(0)

        # Temporal modeling with LSTM
        lstm_out, _ = self.temporal_lstm(spatial_features)  # (B, T, 512)

        # Create attention mask for padded frames (optional)
        if valid_frames is not None:
            # Create mask for attention
            mask = torch.zeros(B, T, dtype=torch.bool, device=x.device)
            for i, vf in enumerate(valid_frames):
                mask[i, :vf] = True
            attn_mask = ~mask
        else:
            attn_mask = None

        # Temporal attention
        attended_features, attention_weights = self.temporal_attention(
            lstm_out, lstm_out, lstm_out, key_padding_mask=attn_mask
        )  # (B, T, 512)

        # Global temporal pooling (weighted by valid frames)
        if valid_frames is not None:
            # Weighted average based on valid frames
            temporal_representation = []
            for i, vf in enumerate(valid_frames):
                temporal_representation.append(
                    torch.mean(attended_features[i, :vf], dim=0)
                )
            temporal_representation = torch.stack(temporal_representation)
        else:
            temporal_representation = torch.mean(attended_features, dim=1)  # (B, 512)

        # Classification
        output = self.classifier(temporal_representation)

        return output

# ----------- Training Utilities -----------
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

def analyze_dataset_composition(dataset):
    """Analyze how many samples were created from sequences of different lengths"""
    print(f"Total samples created: {len(dataset)}")

    # Count class distribution
    original_count = sum(1 for _, label, _ in dataset.samples if label == 0)
    fake_count = sum(1 for _, label, _ in dataset.samples if label == 1)

    print(f"Original samples: {original_count}")
    print(f"Fake samples: {fake_count}")
    print(f"Class ratio (fake/original): {fake_count/max(original_count, 1):.2f}")

    # Analyze batch sizes
    batch_lengths = [len(frames) for frames, _, _ in dataset.samples]
    print(f"Batch length distribution:")
    print(f"  Min: {min(batch_lengths)}")
    print(f"  Max: {max(batch_lengths)}")
    print(f"  Mean: {np.mean(batch_lengths):.1f}")

    # Count how many batches are exactly max_frames
    full_batches = sum(1 for length in batch_lengths if length == dataset.max_frames)
    print(f"Batches with exactly {dataset.max_frames} frames: {full_batches} ({full_batches/len(batch_lengths)*100:.1f}%)")

def train(model, train_loader, val_loader, device, num_epochs=30, checkpoint_path='best_model.pth'):
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 1.5]).to(device))  # Weight for class imbalance
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    scaler = GradScaler()

    best_val_f1 = 0.0
    early_stop_counter = 0
    patience = 7

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, all_preds, all_labels = 0, [], []

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, batch_data in enumerate(progress_bar):
            if len(batch_data) == 3:
                inputs, labels, valid_frames = batch_data
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                valid_frames = valid_frames.to(device, non_blocking=True)
            else:
                inputs, labels = batch_data
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                valid_frames = None

            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs, valid_frames)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

            # Clear cache periodically
            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()

        train_acc, train_prec, train_rec, train_f1 = calculate_metrics(all_labels, all_preds)
        avg_train_loss = train_loss / len(train_loader)

        print(f"Train - Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")

        # Validation phase
        model.eval()
        val_loss, all_preds, all_labels = 0, [], []

        with torch.no_grad():
            for batch_data in tqdm(val_loader, desc="Validating"):
                if len(batch_data) == 3:
                    inputs, labels, valid_frames = batch_data
                    inputs = inputs.to(device, non_blocking=True)
                    labels = labels.to(device, non_blocking=True)
                    valid_frames = valid_frames.to(device, non_blocking=True)
                else:
                    inputs, labels = batch_data
                    inputs = inputs.to(device, non_blocking=True)
                    labels = labels.to(device, non_blocking=True)
                    valid_frames = None

                with autocast():
                    outputs = model(inputs, valid_frames)
                    loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_acc, val_prec, val_rec, val_f1 = calculate_metrics(all_labels, all_labels)
        avg_val_loss = val_loss / len(val_loader)

        print(f"Val - Loss: {avg_val_loss:.4f}, Acc: {val_acc:.4f}, Prec: {val_prec:.4f}, Rec: {val_rec:.4f}, F1: {val_f1:.4f}")
        print("-" * 60)

        scheduler.step(avg_val_loss)

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_f1': val_f1,
            'val_loss': avg_val_loss
        }
        torch.save(checkpoint, f"checkpoint_epoch_{epoch+1}.pth")

        # Save best model based on F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(checkpoint, checkpoint_path)
            print(f"New best model saved with F1: {val_f1:.4f}")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement.")
                break

        # Clear cache
        torch.cuda.empty_cache()

    print(f"Training completed. Best validation F1: {best_val_f1:.4f}")

# ----------- Main Execution -----------
if __name__ == '__main__':
    # Paths - Update these to match your dataset structure
    train_dir = '/root/.cache/kagglehub/datasets/binshilin/poc-final-dataset-vv/versions/1/train'
    test_dir = '/root/.cache/kagglehub/datasets/binshilin/poc-final-dataset-vv/versions/1/test'

    # Enhanced data augmentation for training
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
        ], p=0.7),
        transforms.RandomGrayscale(p=0.05),
        transforms.RandomRotation(10),
        transforms.RandomApply([
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.5))
        ], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
    ])

    # Simple transform for validation
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create datasets with sequential batching
    print("Creating training dataset with sequential batching...")
    train_dataset = FaceSequenceDataset(train_dir, transform=train_transform, max_frames=32, min_frames=16)

    print("Creating validation dataset with sequential batching...")
    val_dataset = FaceSequenceDataset(test_dir, transform=val_transform, max_frames=32, min_frames=16)

    # Analyze dataset composition
    print("\n=== Training Dataset Analysis ===")
    analyze_dataset_composition(train_dataset)
    print("\n=== Validation Dataset Analysis ===")
    analyze_dataset_composition(val_dataset)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=2,  # Adjust based on your GPU memory
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        drop_last=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=2,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Initialize model and training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nUsing device: {device}")

    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    model = SpatialTemporalDeepfakeDetector().to(device)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

    print(f"\nDataLoader info:")
    print(f"Train batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")

    # Start training
    print("\nStarting training...")
    train(model, train_loader, val_loader, device, num_epochs=30)

Creating training dataset with sequential batching...
Scanning dataset in: /root/.cache/kagglehub/datasets/binshilin/poc-final-dataset-vv/versions/1/train
Using max_frames=32, min_frames=16
Processing original class...
Skipping clip_tBhx3g7hXNw_person_001: only 3 frames (< 16)
Skipping clip_lcClO5lHEjA_person_002: only 1 frames (< 16)
Processed 100 original folders, created 204 total batches so far
Finished processing original class: 100 folders
Processing fake class...
Skipping Tina_Shift_0_right_person_002: only 1 frames (< 16)
Skipping Eyelevel_closeup__202508101740_ez9gb_person_003: only 1 frames (< 16)
Processed 100 fake folders, created 484 total batches so far
Finished processing fake class: 100 folders
Creating validation dataset with sequential batching...
Scanning dataset in: /root/.cache/kagglehub/datasets/binshilin/poc-final-dataset-vv/versions/1/test
Using max_frames=32, min_frames=16
Processing original class...
Skipping clip_z0gv5NQPuv4_person_001: only 1 frames (< 16)
F

100%|██████████| 35.1M/35.1M [00:00<00:00, 67.1MB/s]


Loaded pretrained weights for efficientnet-b2
Total parameters: 12,807,812
Trainable parameters: 12,807,812

DataLoader info:
Train batches: 242
Validation batches: 23

Starting training...


Epoch 1/30 - Training: 100%|██████████| 242/242 [02:32<00:00,  1.59it/s, loss=0.0480]


Train - Loss: 0.5717, Acc: 0.6715, F1: 0.7602


...

Train - Loss: 0.3380, Acc: 0.8574, F1: 0.8774


Validating: 100%|██████████| 23/23 [00:04<00:00,  5.49it/s]


Val - Loss: 0.1850, Acc: 1.0000, Prec: 1.0000, Rec: 1.0000, F1: 1.0000
------------------------------------------------------------


Epoch 3/30 - Training:   9%|▉         | 22/242 [00:14<02:22,  1.55it/s, loss=0.0852]


KeyboardInterrupt: 

In [ ]:
# Inference batch wise split

In [5]:
! pip install kagglehub opencv-python mtcnn deep_sort_realtime opencv-python mtcnn keras-facenet numpy scipy facenet-pytorch

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=effd08ab9d7c9d5c56a5ca2f18abe564002ac947cbaf1d19d6034d1dcfa42bd1
  Stored in directory: /root/.cache/pip/wheels/05/b0/f5/19ac49fedc10b1df3ee56b096edbcfa39d45794fccc6bcdbbf
Successfully built keras-facenet


In [10]:
import cv2
import os
import numpy as np
import torch
import torch.nn as nn
from datetime import datetime
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine
import shutil
import tempfile
from PIL import Image
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import warnings
warnings.filterwarnings("ignore")

# ==== Model Definition (Same as training script) ====
class SpatialTemporalDeepfakeDetector(nn.Module):
    def __init__(self, num_classes=2, max_frames=32, feature_dim=1408):
        super().__init__()

        # Spatial feature extractor
        self.spatial_encoder = EfficientNet.from_pretrained('efficientnet-b2')
        self.spatial_encoder._fc = nn.Identity()

        # Spatial feature projection
        self.spatial_projection = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Temporal modeling with LSTM + Transformer
        self.temporal_lstm = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )

        # Temporal attention
        self.temporal_attention = nn.MultiheadAttention(
            embed_dim=512,  # bidirectional LSTM output
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )

        # Positional encoding for temporal sequences
        self.pos_encoding = nn.Parameter(torch.randn(max_frames, 512))

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x, valid_frames=None):  # x: (B, T, C, H, W)
        B, T, C, H, W = x.shape

        # Extract spatial features for each frame
        x_reshaped = x.view(B * T, C, H, W)
        spatial_features = self.spatial_encoder.extract_features(x_reshaped)  # (B*T, 1408, h, w)

        # Project spatial features
        spatial_features = self.spatial_projection(spatial_features)  # (B*T, 512)
        spatial_features = spatial_features.view(B, T, -1)  # (B, T, 512)

        # Add positional encoding
        spatial_features = spatial_features + self.pos_encoding[:T].unsqueeze(0)

        # Temporal modeling with LSTM
        lstm_out, _ = self.temporal_lstm(spatial_features)  # (B, T, 512)

        # Create attention mask for padded frames (optional)
        if valid_frames is not None:
            # Create mask for attention
            mask = torch.zeros(B, T, dtype=torch.bool, device=x.device)
            for i, vf in enumerate(valid_frames):
                mask[i, :vf] = True
            attn_mask = ~mask
        else:
            attn_mask = None

        # Temporal attention
        attended_features, attention_weights = self.temporal_attention(
            lstm_out, lstm_out, lstm_out, key_padding_mask=attn_mask
        )  # (B, T, 512)

        # Global temporal pooling (weighted by valid frames)
        if valid_frames is not None:
            # Weighted average based on valid frames
            temporal_representation = []
            for i, vf in enumerate(valid_frames):
                temporal_representation.append(
                    torch.mean(attended_features[i, :vf], dim=0)
                )
            temporal_representation = torch.stack(temporal_representation)
        else:
            temporal_representation = torch.mean(attended_features, dim=1)  # (B, 512)

        # Classification
        output = self.classifier(temporal_representation)

        return output

# ==== Face Extraction Class ====
class FaceExtractor:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.detector = MTCNN(keep_all=True, device=device)
        self.embedder = InceptionResnetV1(pretrained='vggface2').eval().to(device)

        # Parameters
        self.max_cosine_dist = 0.5
        self.max_age = 30  # max frame age to retain a track
        self.sequence_length = 32  # Match model's max_frames
        self.max_gap_sec = 0.5
        self.sample_interval_sec = 0.1

    def extract_faces_from_video(self, video_path, output_dir=None):
        """Extract face sequences from a video"""
        if output_dir is None:
            output_dir = tempfile.mkdtemp(prefix='face_extraction_')

        os.makedirs(output_dir, exist_ok=True)
        print(f"🎬 Processing video: {video_path}")
        print(f"📁 Output directory: {output_dir}")

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"❌ Failed to open {video_path}")

        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0:
            fps = 25.0
        frame_interval = int(round(fps * self.sample_interval_sec))

        # Reset state
        tracks = {}
        track_frames = {}
        next_track_id = 1
        frame_num = 0
        processed_frame_count = 0

        print(f"🔍 Extracting faces at {1/self.sample_interval_sec:.1f} FPS...")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_num += 1

            if frame_num % frame_interval != 0:
                continue

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # seconds

            # Detect faces
            boxes, probs = self.detector.detect(rgb)
            if boxes is None:
                continue

            new_faces = []
            for i, (box, prob) in enumerate(zip(boxes, probs)):
                if prob < 0.99:
                    continue
                x1, y1, x2, y2 = map(int, box)
                margin = 10
                x1 = max(0, x1 - margin)
                y1 = max(0, y1 - margin)
                x2 = min(frame.shape[1], x2 + margin)
                y2 = min(frame.shape[0], y2 + margin)

                face_crop = rgb[y1:y2, x1:x2]
                if face_crop.size == 0 or face_crop.shape[0] < 60 or face_crop.shape[1] < 60:
                    continue

                resized = cv2.resize(face_crop, (160, 160))
                face_tensor = torch.tensor(resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0
                face_tensor = face_tensor.to(self.device)

                with torch.no_grad():
                    embedding = self.embedder(face_tensor).cpu().numpy()[0]

                new_faces.append((embedding, face_crop))

            # Track assignment
            assigned = []
            for emb, crop in new_faces:
                best_match = None
                best_dist = float('inf')
                for track_id, data in tracks.items():
                    dist = cosine(emb, data['embedding'])
                    if dist < best_dist and dist < self.max_cosine_dist:
                        best_dist = dist
                        best_match = track_id

                if best_match is not None:
                    tracks[best_match]['embedding'] = emb
                    tracks[best_match]['last_frame'] = frame_num
                    assigned.append((best_match, crop))
                else:
                    track_id = next_track_id
                    next_track_id += 1
                    tracks[track_id] = {'embedding': emb, 'last_frame': frame_num}
                    track_frames[track_id] = []
                    assigned.append((track_id, crop))

            # Save frames
            for track_id, crop in assigned:
                save_path = os.path.join(output_dir, f'person_{track_id:03d}')
                os.makedirs(save_path, exist_ok=True)
                processed_frame_count += 1
                filename = f'frame_{processed_frame_count:04d}_{frame_time:.2f}s.jpg'
                filepath = os.path.join(save_path, filename)
                cv2.imwrite(filepath, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
                track_frames[track_id].append((frame_time, filepath))

            # Clean old tracks
            tracks = {k: v for k, v in tracks.items() if frame_num - v['last_frame'] <= self.max_age}

        cap.release()

        # Post-process sequences
        self._post_process_sequences(track_frames)

        print(f"✅ Face extraction complete!")
        print(f"📊 Found {len(track_frames)} face tracks")
        for track_id, frames in track_frames.items():
            print(f"   Person {track_id:03d}: {len(frames)} frames")

        return output_dir, track_frames

    def _post_process_sequences(self, track_frames):
        """Pad sequences for RNN/LSTM processing"""
        print("🔧 Post-processing sequences...")

        for track_id, frames in track_frames.items():
            if not frames:
                continue

            # Sort original frames by timestamp
            frames.sort(key=lambda x: x[0])
            save_dir = os.path.dirname(frames[0][1])
            valid_frames = [frames[0]]  # Start with first frame

            # Fill time gaps with duplicates of previous frame
            for i in range(1, len(frames)):
                prev_time, prev_path = valid_frames[-1]
                curr_time, curr_path = frames[i]
                time_diff = curr_time - prev_time

                if time_diff <= self.max_gap_sec:
                    # Compute how many frames are missing based on sample interval
                    n_missing = int(round(time_diff / self.sample_interval_sec)) - 1
                    for m in range(n_missing):
                        pad_time = prev_time + (m + 1) * self.sample_interval_sec
                        pad_path = os.path.join(save_dir, f'frame_fill_tmp_{pad_time:.2f}s.jpg')
                        shutil.copy(prev_path, pad_path)
                        valid_frames.append((pad_time, pad_path))

                    valid_frames.append((curr_time, curr_path))
                else:
                    break  # Stop this track if time jump too large

            # Pad end of sequence if needed to reach required length
            if len(valid_frames) < self.sequence_length:
                last_time, last_path = valid_frames[-1]
                for i in range(len(valid_frames), self.sequence_length):
                    pad_time = last_time + (i - len(valid_frames) + 1) * self.sample_interval_sec
                    pad_path = os.path.join(save_dir, f'frame_pad_tmp_{pad_time:.2f}s.jpg')
                    shutil.copy(last_path, pad_path)
                    valid_frames.append((pad_time, pad_path))

            # Final sort and renaming to guarantee temporal order
            valid_frames.sort(key=lambda x: x[0])
            for i, (t, old_path) in enumerate(valid_frames):
                new_filename = f'frame_{i+1:04d}_{t:.2f}s.jpg'
                new_path = os.path.join(save_dir, new_filename)
                if old_path != new_path:  # Only rename if different
                    os.rename(old_path, new_path)

# ==== Inference Class ====
class DeepfakeInference:
    def __init__(self, model_path, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.model = SpatialTemporalDeepfakeDetector().to(device)

        # Load trained model
        print(f"📦 Loading model from: {model_path}")
        checkpoint = torch.load(model_path, map_location=device)
        if 'model_state_dict' in checkpoint:
            self.model.load_state_dict(checkpoint['model_state_dict'])
        else:
            self.model.load_state_dict(checkpoint)

        self.model.eval()

        # Image preprocessing
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        print(f"✅ Model loaded successfully on {device}")

    def predict_sequence_batches(self, sequence_dir, max_frames=32, min_frames=16):
        """Predict on a face sequence directory using batches like training"""
        try:
            # Get all frame files
            frame_files = []
            for ext in ['*.jpg', '*.jpeg', '*.png']:
                import glob
                frame_files.extend(glob.glob(os.path.join(sequence_dir, ext)))

            frame_files = sorted(frame_files)

            if len(frame_files) == 0:
                print(f"⚠️ No frames found in {sequence_dir}")
                return []

            print(f"📸 Found {len(frame_files)} frames in {sequence_dir}")

            # Create batches like in training
            batches_results = []
            total_frames = len(frame_files)

            if total_frames < min_frames:
                print(f"⚠️ Skipping {sequence_dir}: only {total_frames} frames (< {min_frames})")
                return []

            # Create non-overlapping batches (same as training)
            stride = max_frames  # Non-overlapping batches
            start_idx = 0
            batch_num = 1

            while start_idx < total_frames:
                end_idx = min(start_idx + max_frames, total_frames)
                batch_frame_files = frame_files[start_idx:end_idx]

                # Only process batches that meet minimum frame requirement
                if len(batch_frame_files) >= min_frames:
                    print(f"  🔄 Processing batch {batch_num}: frames {start_idx+1} to {end_idx} ({len(batch_frame_files)} frames)")

                    # Load and preprocess frames for this batch
                    images = []
                    successful_loads = 0

                    for frame_path in batch_frame_files:
                        try:
                            img = Image.open(frame_path).convert('RGB')
                            img = self.transform(img)
                            images.append(img)
                            successful_loads += 1
                        except Exception as e:
                            print(f"⚠️ Error loading {frame_path}: {e}")
                            # Use black image as fallback
                            fallback_img = Image.new('RGB', (224, 224), (0, 0, 0))
                            img = self.transform(fallback_img)
                            images.append(img)

                    if len(images) == 0:
                        print(f"⚠️ No valid frames loaded for batch {batch_num}")
                        start_idx += stride
                        continue

                    # Pad sequence to max_frames if needed (same as training)
                    while len(images) < max_frames:
                        images.append(images[-1].clone())  # Duplicate last frame

                    # Convert to tensor and add batch dimension
                    sequence = torch.stack(images).unsqueeze(0).to(self.device)  # (1, T, C, H, W)
                    valid_frames = torch.tensor([successful_loads]).to(self.device)

                    # Predict
                    with torch.no_grad():
                        outputs = self.model(sequence, valid_frames)
                        probabilities = torch.softmax(outputs, dim=1)
                        prediction = torch.argmax(outputs, dim=1)
                        confidence = torch.max(probabilities, dim=1)[0]

                    batch_result = {
                        'batch_number': batch_num,
                        'frame_range': f"{start_idx+1}-{end_idx}",
                        'prediction': 'FAKE' if prediction.item() == 1 else 'REAL',
                        'confidence': confidence.item(),
                        'fake_probability': probabilities[0, 1].item(),
                        'real_probability': probabilities[0, 0].item(),
                        'frames_in_batch': len(batch_frame_files),
                        'valid_frames': successful_loads,
                        'start_frame': start_idx + 1,
                        'end_frame': end_idx
                    }

                    batches_results.append(batch_result)

                    # Print batch result immediately
                    print(f"    📊 Batch {batch_num} Result: {batch_result['prediction']} "
                          f"(confidence: {batch_result['confidence']:.3f}, "
                          f"fake_prob: {batch_result['fake_probability']:.3f})")

                    batch_num += 1

                start_idx += stride

                # If remaining frames are less than min_frames, break
                if total_frames - start_idx < min_frames:
                    break

            return batches_results

        except Exception as e:
            print(f"❌ Error in predict_sequence_batches: {str(e)}")
            import traceback
            traceback.print_exc()
            return []

# ==== Main Pipeline Class ====
class DeepfakeDetectionPipeline:
    def __init__(self, model_path, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.face_extractor = FaceExtractor(device)
        self.inference = DeepfakeInference(model_path, device)

    def process_video(self, video_path, cleanup_temp=True):
        """Complete pipeline: extract faces and run inference"""
        print(f"🚀 Starting deepfake detection pipeline...")
        print(f"📹 Video: {video_path}")

        # Extract faces
        temp_dir, track_frames = self.face_extractor.extract_faces_from_video(video_path)

        if not track_frames:
            print("❌ No faces found in video")
            return None

        # Run inference on each face track
        results = {}
        print(f"\n🔮 Running batch inference on {len(track_frames)} face tracks...")

        for track_id, frames in track_frames.items():
            person_dir = os.path.join(temp_dir, f'person_{track_id:03d}')

            if os.path.exists(person_dir):
                print(f"\n👤 Processing Person {track_id:03d}:")
                batch_results = self.inference.predict_sequence_batches(person_dir)

                if batch_results:
                    results[f'person_{track_id:03d}'] = {
                        'batches': batch_results,
                        'total_batches': len(batch_results)
                    }

                    # Print summary for this person
                    fake_count = sum(1 for batch in batch_results if batch['prediction'] == 'FAKE')
                    real_count = sum(1 for batch in batch_results if batch['prediction'] == 'REAL')
                    avg_confidence = sum(batch['confidence'] for batch in batch_results) / len(batch_results)
                    avg_fake_prob = sum(batch['fake_probability'] for batch in batch_results) / len(batch_results)

                    print(f"  📈 Person {track_id:03d} Summary:")
                    print(f"     Total batches: {len(batch_results)}")
                    print(f"     FAKE batches: {fake_count}")
                    print(f"     REAL batches: {real_count}")
                    print(f"     Average confidence: {avg_confidence:.3f}")
                    print(f"     Average fake probability: {avg_fake_prob:.3f}")

                    # Determine overall prediction for this person
                    person_prediction = "FAKE" if fake_count > real_count else "REAL"
                    print(f"     🎯 Person prediction: {person_prediction}")

                else:
                    print(f"   ❌ Person {track_id:03d}: No valid batches processed")

        # Cleanup temporary directory
        if cleanup_temp:
            try:
                shutil.rmtree(temp_dir)
                print(f"\n🧹 Cleaned up temporary directory")
            except Exception as e:
                print(f"⚠️ Could not cleanup temp directory: {e}")
        else:
            print(f"\n📁 Temporary files kept in: {temp_dir}")

        return results

    def get_overall_prediction(self, results):
        """Get overall video prediction based on all face tracks and their batches"""
        if not results:
            return "UNKNOWN", 0.0

        total_fake_batches = 0
        total_real_batches = 0
        total_confidence = 0
        total_batches = 0

        for person, person_data in results.items():
            if 'batches' in person_data:
                for batch in person_data['batches']:
                    if batch['prediction'] == 'FAKE':
                        total_fake_batches += 1
                    else:
                        total_real_batches += 1

                    total_confidence += batch['confidence']
                    total_batches += 1

        if total_batches == 0:
            return "UNKNOWN", 0.0

        avg_confidence = total_confidence / total_batches

        # If majority of batches are fake, classify as fake
        if total_fake_batches > total_real_batches:
            return "FAKE", avg_confidence
        else:
            return "REAL", avg_confidence

# ==== Usage Example ====
def main():
    # Configuration
    MODEL_PATH = '/content/checkpoint_epoch_2.pth'  # Path to your trained model
    VIDEO_PATH = '/content/Two_senior_politicians_202508101747_mlk3j.mp4'  # Path to input video

    try:
        # Initialize pipeline
        pipeline = DeepfakeDetectionPipeline(MODEL_PATH)

        # Process video
        results = pipeline.process_video(VIDEO_PATH, cleanup_temp=True)

        if results:
            print(f"\n📊 DETAILED BATCH RESULTS:")
            print("=" * 60)

            for person, person_data in results.items():
                if 'batches' in person_data:
                    print(f"\n{person.upper()}:")
                    print(f"  Total Batches: {person_data['total_batches']}")
                    print("  " + "-" * 40)

                    for batch in person_data['batches']:
                        print(f"  Batch {batch['batch_number']} (frames {batch['frame_range']}):")
                        print(f"    Prediction: {batch['prediction']}")
                        print(f"    Confidence: {batch['confidence']:.3f}")
                        print(f"    Real Probability: {batch['real_probability']:.3f}")
                        print(f"    Fake Probability: {batch['fake_probability']:.3f}")
                        print(f"    Frames in batch: {batch['frames_in_batch']}")
                        print(f"    Valid frames: {batch['valid_frames']}")
                        print()

                    # Person-level summary
                    fake_batches = sum(1 for batch in person_data['batches'] if batch['prediction'] == 'FAKE')
                    real_batches = sum(1 for batch in person_data['batches'] if batch['prediction'] == 'REAL')
                    avg_conf = sum(batch['confidence'] for batch in person_data['batches']) / len(person_data['batches'])

                    print(f"  🎯 {person} SUMMARY:")
                    print(f"    FAKE batches: {fake_batches}/{person_data['total_batches']}")
                    print(f"    REAL batches: {real_batches}/{person_data['total_batches']}")
                    print(f"    Average confidence: {avg_conf:.3f}")
                    person_pred = "FAKE" if fake_batches > real_batches else "REAL"
                    print(f"    Overall person prediction: {person_pred}")
                    print()

            # Overall video prediction
            overall_pred, overall_conf = pipeline.get_overall_prediction(results)

            # Count total batches across all people
            total_batches = sum(person_data['total_batches'] for person_data in results.values() if 'batches' in person_data)
            total_fake = sum(sum(1 for batch in person_data['batches'] if batch['prediction'] == 'FAKE')
                           for person_data in results.values() if 'batches' in person_data)
            total_real = total_batches - total_fake

            print("=" * 60)
            print(f"🎯 OVERALL VIDEO PREDICTION: {overall_pred}")
            print(f"   Average confidence: {overall_conf:.3f}")
            print(f"   Total batches analyzed: {total_batches}")
            print(f"   FAKE batches: {total_fake}")
            print(f"   REAL batches: {total_real}")
            print(f"   Fake ratio: {total_fake/max(total_batches, 1):.3f}")
            print("=" * 60)

        else:
            print("❌ No results obtained from video processing")

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

# ==== Standalone Function for Easy Use ====
def detect_deepfake(video_path, model_path, cleanup_temp=True):
    """
    Standalone function to detect deepfakes in a video

    Args:
        video_path (str): Path to the input video
        model_path (str): Path to the trained model (.pth file)
        cleanup_temp (bool): Whether to cleanup temporary files

    Returns:
        dict: Detection results for each face track found
    """
    pipeline = DeepfakeDetectionPipeline(model_path)
    return pipeline.process_video(video_path, cleanup_temp)

Loaded pretrained weights for efficientnet-b2
📦 Loading model from: /content/checkpoint_epoch_2.pth
✅ Model loaded successfully on cuda
🚀 Starting deepfake detection pipeline...
📹 Video: /content/Two_senior_politicians_202508101747_mlk3j.mp4
🎬 Processing video: /content/Two_senior_politicians_202508101747_mlk3j.mp4
📁 Output directory: /tmp/face_extraction_elt94xsx
🔍 Extracting faces at 10.0 FPS...
🔧 Post-processing sequences...
✅ Face extraction complete!
📊 Found 2 face tracks
   Person 001: 96 frames
   Person 002: 92 frames

🔮 Running batch inference on 2 face tracks...

👤 Processing Person 001:
📸 Found 96 frames in /tmp/face_extraction_elt94xsx/person_001
  🔄 Processing batch 1: frames 1 to 32 (32 frames)
    📊 Batch 1 Result: FAKE (confidence: 0.591, fake_prob: 0.591)
  🔄 Processing batch 2: frames 33 to 64 (32 frames)
    📊 Batch 2 Result: FAKE (confidence: 0.925, fake_prob: 0.925)
  🔄 Processing batch 3: frames 65 to 96 (32 frames)
    📊 Batch 3 Result: FAKE (confidence: 0.868, f

In [11]:
# Initialize pipeline
pipeline = DeepfakeDetectionPipeline('best_model.pth')

# Process video
results = pipeline.process_video('path/to/video.mp4')

# Get overall prediction
overall_pred, confidence = pipeline.get_overall_prediction(results)
print(f"Video is {overall_pred} with {confidence:.3f} confidence")

Loaded pretrained weights for efficientnet-b2


ValueError: max() iterable argument is empty